### Data tunning

In [13]:
import pandas as pd
import numpy as np
import random

In [14]:
datos = pd.read_csv('/content/2_MOCK_DATA.csv')
datos = datos.sort_values(by=['hora']).reset_index(drop=True)

In [15]:
# Creo la columna 'consumo_del_dia' con datos coherentes
consumo = []
for i in range(1000):
  consumo.append(i/random.randrange(30,38) + i*0.01)
consumo.sort()
consumo = pd.DataFrame(consumo)
consumo = round(consumo,4)
datos = pd.concat([datos, consumo], axis=1)
datos.rename(columns = {0:'consumo_del_dia'}, inplace = True)

In [16]:
# Adecúo el campo 'hora' para que quede cada 30 segundos (como dice la letra)
datos['fecha2'] = '2022-07-14'
datos['hora'] = pd.date_range("11:00:00", "19:19:30", freq="30S").strftime('%H:%M:%S')
datos['fecha_hora'] = datos["fecha2"] + ' ' + datos["hora"]
datos = datos.loc[:, datos.columns != 'fecha']
datos = datos.loc[:, datos.columns != 'fecha2']
datos = datos.loc[:, datos.columns != 'hora']

In [17]:
# Creo la columna 'bateria_restante' con los datos de bateria1 y bateria2
bateria_1 = datos['bateria1']
bateria_1 = pd.DataFrame(bateria_1).sort_values(by='bateria1', ascending=False).reset_index(drop=True).tail(500)
bateria_1 = list(bateria_1['bateria1'])
bateria_2 = datos['bateria2']
bateria_2 = pd.DataFrame(bateria_2).sort_values(by='bateria2', ascending=False).reset_index(drop=True).head(500)
bateria_2 = list(bateria_2['bateria2'])
bateria_restante = bateria_1 + bateria_2
bateria_restante = pd.DataFrame(bateria_restante).reset_index(drop=True)
datos['bateria'] = bateria_restante
datos = datos.loc[:, datos.columns != 'bateria1']
datos = datos.loc[:, datos.columns != 'bateria2']

In [18]:
# Creo la columna 'dist_supercharger' con los datos de dist_supercharger1 y dist_supercharger2
dist_1 = pd.DataFrame(datos['dist_supercharger1'])
dist_1 = dist_1.sort_values(by='dist_supercharger1', ascending=False)
dist_1 = dist_1.tail(500).reset_index(drop=True)
dist_1 = list(dist_1['dist_supercharger1'])
dist_2 = pd.DataFrame(datos['dist_supercharger2'])
dist_2 = dist_2.sort_values(by='dist_supercharger2', ascending=False)
dist_2 = dist_2.head(500).reset_index(drop=True)
dist_2 = list(dist_2['dist_supercharger2'])
dist_supercharger = dist_1 + dist_2
datos['dist_supercharger'] = dist_supercharger
datos = datos.loc[:, datos.columns != 'dist_supercharger1']
datos = datos.loc[:, datos.columns != 'dist_supercharger2']

In [19]:
# Reordeno las columnas
datos = datos[['id_vehiculo', 'fecha_hora', 'ondas', 'temp_motor', 'vibracion', 'presion',
               'humedad','voltaje', 'velocidad','consumo_del_dia','bateria', 'dist_supercharger']]

In [20]:
consumo = []
for i in range(500,1000):
  consumo.append(datos.at[i,'consumo_del_dia'])

# Como el auto esta apagado al cargarse, los sensores no registran información
for i in range(500,570):
  datos.at[i,'ondas']=np.nan
  datos.at[i,'temp_motor']=np.nan
  datos.at[i,'vibracion']=np.nan
  datos.at[i,'presion']=np.nan
  datos.at[i,'humedad']=np.nan
  datos.at[i,'voltaje']=np.nan
  datos.at[i,'velocidad']=np.nan
  datos.at[i,'bateria']=np.nan
  datos.at[i,'consumo_del_dia']=np.nan
  datos.at[i,'dist_supercharger']=np.nan

# Para que al arrancar el auto, consumo_del_dia empiece desde el ultimo valor
for i,z in enumerate(range(570,1000)):
  datos.at[z, 'consumo_del_dia'] = consumo[i]

datos          

,id_vehiculo,fecha_hora,ondas,temp_motor,vibracion,presion,humedad,voltaje,velocidad,consumo_del_dia,bateria,dist_supercharger
0,24596,2022-07-14 11:00:00,0.749,88.0,0.811,2.42,0.92,345.8,54.4,0.0000,60.0,246.8
1,24596,2022-07-14 11:00:30,0.818,85.1,0.837,3.55,0.79,436.9,71.3,0.0370,60.0,246.4
2,24596,2022-07-14 11:01:00,0.738,98.2,0.761,3.92,0.91,380.0,40.4,0.0867,59.9,246.2
3,24596,2022-07-14 11:01:30,0.894,89.9,1.000,3.71,0.81,402.2,79.9,0.1238,59.9,246.1
4,24596,2022-07-14 11:02:00,0.756,96.2,0.774,3.92,0.95,339.7,52.7,0.1733,59.8,245.1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,24596,2022-07-14 19:17:30,0.802,92.7,0.972,2.16,0.95,417.3,69.3,36.8789,70.2,607.4
996,24596,2022-07-14 19:18:00,0.740,99.0,0.879,2.74,0.86,434.0,59.4,36.9176,70.1,607.3
997,24596,2022-07-14 19:18:30,0.706,88.8,0.928,3.57,0.96,373.7,42.2,36.9982,70.0,606.9
998,24596,2022-07-14 19:19:00,0.963,90.0,0.812,3.90,0.98,392.6,68.3,37.0286,70.0,606.6


### CSV

In [ ]:
datos.to_csv('/content/data_to_elastic.csv')

### JSON

In [21]:
datos.to_json("./4_data_to_elastic.json", orient="records", lines=True)